In [71]:
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch
from storage.query_storage import QueryStorage
from tqdm import tqdm
import matplotlib.pyplot as plt
import sklearn
from collections import OrderedDict
import os
import pickle

enc_len = 50
num_actions = 21
mol_enc =   OrderedDict([('A', 0), ('R', 1), ('N', 2), ('D', 3), ('C', 4), ('E', 5), ('Q', 6), ('G', 7), ('H', 8), ('I', 9), ('L', 10), ('K', 11), ('M', 12), ('F', 13), ('P', 14), ('S', 15), ('T', 16), ('W', 17), ('Y', 18), ('V', 19), ('>', 20)])

def seq_to_enc(seq):
    enc = [None for i in range(enc_len)]
    for i in range(enc_len):
        if i < len(seq):
            enc[i] = mol_enc[seq[i]]
        else:
            enc[i] = 20
    
    return F.one_hot(torch.tensor(enc), num_classes=num_actions).numpy()

def get_AMP_data(mode):

    if mode == "val":
        df = pd.read_csv("data/210820_v0.24.1_rf_not-alibicani.csv")
    elif mode == "test":
        df = pd.read_csv("data/210820_v0.24.1_rf_alibicani.csv")
    else:
        raise NotImplementedError
 
#     df = df[(df.sequence.map(len) >= 15)]
    filtered_seq = df.sequence.map(lambda x : 'C' not in x and len(x) >= 15 and len(x) <= 50)
    df = df[filtered_seq]
    
    df.sequence = df.sequence.map(seq_to_enc)

    storage = QueryStorage(df.sequence.shape[0], df.sequence.shape[1:])

    storage.mols = torch.tensor(df.sequence.to_list())

    storage.scores = torch.tensor(df.is_amp.to_list()).long()

    return storage




In [72]:
data = get_AMP_data("val")

In [73]:
data.mols

tensor([[[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[1, 0, 0,  ..., 0, 0, 0],
         [1, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        ...,

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 1]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0,

In [74]:
import random
from tqdm import tqdm

num_seeds = 10
scores = []


for s in tqdm(range(num_seeds)):
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(random_state=s, n_estimators=128)
    model.fit(seq, value)
    
    max_scores = []

    for qq in range(8):
        mol_probs = []
        mols = []
        for i in range(250):
            rand_len = random.randint(1, 50)

            rand_mol = [random.randint(0, 19) for i in range(rand_len)]
            rand_mol += (50 - len(rand_mol))*[20]
            rand_mol = torch.tensor(rand_mol)
            rand_mol = torch.nn.functional.one_hot(rand_mol, num_classes=21).flatten(-2, -1)
            mols.append(rand_mol)


        mol_probs = model.predict_proba(torch.stack(mols, 0).numpy())
        
        mol_probs = mol_probs[:, 1]

#         print("Average score:", mol_probs.mean())

#         print("Cumulative Max Score:", mol_probs.max())


        max_scores.append(mol_probs.max())



    scores.append(max_scores)

  0%|          | 0/10 [00:00<?, ?it/s]


NameError: name 'seq' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=128)


seq = data.mols.flatten(start_dim=-2, end_dim=-1)
value = data.scores
model.fit(seq, value)

In [75]:
exp_dir = './logs/logs_AMP-v0/DEBUG-REINFORCE-True_73__09-09_12-02-49'

In [76]:
file = open(os.path.join(exp_dir, 'queried_mols.pkl'), "rb")

queried_data = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: './logs/logs_AMP-v0/DEBUG-REINFORCE-True_73__09-09_12-02-49/queried_mols.pkl'

In [25]:
from config import AMP_configs
from config.default import DEFAULT_CONFIG
from config import debug_configs
from utils.helpers import merge_dicts

config_name = "debug_REINFORCE_True"
    
if config_name == "DEFAULT":
    config = DEFAULT_CONFIG
elif "debug" in config_name:
    debug_config = getattr(debug_configs, config_name)
    config = merge_dicts(DEFAULT_CONFIG, debug_config)
else:
    amp_config = getattr(AMP_configs, config_name)
    config = merge_dicts(DEFAULT_CONFIG, amp_config)

In [27]:
from utils.helpers import get_proxy_oracle_model

proxy_oracle_model = get_proxy_oracle_model(config)

In [30]:
proxy_oracle_model.fit(queried_data.seq, queried_data.pred_prob)

ValueError: setting an array element with a sequence.

In [41]:
np_queried_data = np.array(queried_data.seq.tolist())
np_queried_data = np_queried_data.reshape(np_queried_data.shape[0], -1)

np_pred_prob = np.array(queried_data.pred_prob.tolist())

In [42]:
proxy_oracle_model.fit(np_queried_data, np_pred_prob)

KNeighborsRegressor(n_neighbors=9)

In [52]:
res = proxy_oracle_model.predict(data.mols.flatten(-2, -1))


res.mean()

0.15447284709630302

In [68]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score


class regressor():
    def __init__(self, seq, value):
        super(regressor, self).__init__()
        self.seq = seq
        self.value = value

    def to_fit(self, model):
        return model.fit(self.seq, self.value)

    def to_predict(self, model, seq):
        return model.predict(seq)

    def get_accuracy(self, model, test_seq, test_value):
        pred = self.to_predict(model, test_seq)
        score = r2_score(test_value, pred)
        return score


def XGBoost(learning_rate = 0.1, max_depth = 85, n_estimators = 500):
    model = GradientBoostingRegressor(n_estimators = n_estimators,
                                      max_depth = max_depth,
                                      learning_rate = learning_rate)
    return model

def XGB():
    model = xgb.XGBRegressor(objective ='reg:squarederror',
                             colsample_bytree = 0.3,
                             learning_rate = 0.1,
                             max_depth = 85, alpha = 5)
    return model

def RFR(max_depth = 50, max_features = "log2", n_estimators = 100):
    model = RandomForestRegressor(max_depth = max_depth,
                                  max_features = max_features,
                                  n_estimators = n_estimators)
    return model

def KNN(neighbors = 10):
    model = KNeighborsRegressor(n_neighbors = neighbors)
    return model

def RR():
    model = Ridge(alpha = 1.0)
    return model

In [69]:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

def fit_proxy(X, y, kfld=True):
    """
    This function takes in the all data sampled until now, and
    fits a proxy regressor to emulate the true oracle
    """
#     X = np.array(self.dataset.obs).reshape(-1, 46*21)
#     y = np.array(self.dataset.pred_prob).reshape(-1,)

    def kFoldCV(models, X, y, k=5):
        """
        Perform k-fold validation, return average score
        """
        kf = KFold(n_splits=k,  shuffle=True) #TODO
        kf.get_n_splits(X)

        accuracy = np.zeros((k, len(models)))
        i = 0
        for train_index, test_index in kf.split(X):
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]

            c = regressor(X_train, np.squeeze(y_train))

            for n, m in enumerate(models):
                accuracy[i, n] = c.get_accuracy(c.to_fit(m), X_test, np.squeeze(y_test))
                print(accuracy)
            i += 1
        print("Accuracy array")
        print(accuracy)
        return np.average(accuracy, axis = 0)

    def test_train(models, X, y, test_size = 0.3, shuffle = True):
        proxylist = []
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=0, shuffle=shuffle)

        print("Shape: ", X_train.shape, y_train.shape, X_test.shape)

        c = regressor(X_train, y_train)

        for n, m in enumerate(models):
            pr = c.to_fit(m)
            ac = c.get_accuracy(pr, X_test, y_test)
            proxylist.append((pr, ac))

        return proxylist

#     models = np.array([KNN(), RFR(), XGB(), XGBoost()])
    models = np.array([XGBoost()])

    if kfld:
        accuracy = kFoldCV(models=models, X=X, y=y)
        print("Final accuracy: ", accuracy)
        print(accuracy > 0.5)

        # Select all proxy models with accuracy > threshold
        proxy = models[accuracy > 0.5]
        m = []
        for p in proxy:
            # Tune regressor on all data seen so far
            m.append(regressor(X, y).to_fit(p))
        return m
    else:
        return test_train(models=models, X=X, y=y)

In [70]:
fit_proxy(np_queried_data, np_pred_prob)

KeyboardInterrupt: 